In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import tensorflow

In [2]:
import keras

In [8]:
pip install Keras

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install keras.utils

Processing ./.cache/pip/wheels/d0/dd/3b/493952a5240d486a83805d65360dedadbadeae71d25e2c877f/keras_utils-1.0.13-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import tensorflow as tf

from keras import layers
from keras.layers import Input,Dense,Activation,ZeroPadding2D,BatchNormalization,Flatten,Conv2D
from keras.layers import AveragePooling2D,MaxPooling2D,Dropout,GlobalMaxPooling2D,GlobalAveragePooling2D
from keras.utils import np_utils
from tensorflow.python.keras.utils.layer_utils import print_summary
from tensorflow.python.keras.utils.vis_utils import model_to_dot

#for csv to dataframe
import pandas as pd
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import keras.backend as K

In [2]:
import pandas as pd

data=pd.read_csv("/home/rupeek/Downloads/data.csv")
#print(data.head())
dataset=np.array(data)
np.random.shuffle(dataset)
X=dataset
Y=dataset
X=X[:, 0:1024]
Y=Y[:, 1024]

X_train=X[0:70000, :]
X_train=X_train/255.
X_test=X[70000:72001, :]
X_test=X_test/255.

#Reshape
Y=Y.reshape(Y.shape[0],1)
Y_train=Y[0:70000,:]
Y_train=Y_train.T
Y_test=Y[70000:72001,:]
Y_test=Y_test.T

In [3]:
print("Number of training examples=" + str(X_train.shape[0]))
print("Number of test examples=" + str(X_test.shape[0]))
print("X_train shape=" + str(X_train.shape))
print("Y_train shape=" + str(Y_train.shape))
print("X_test shape=" + str(X_test.shape))
print("Y_test shape=" + str(Y_test.shape))

Number of training examples=70000
Number of test examples=2000
X_train shape=(70000, 1024)
Y_train shape=(1, 70000)
X_test shape=(2000, 1024)
Y_test shape=(1, 2000)


In [4]:
image_x=32
image_y=32

train_y=np_utils.to_categorical(Y_train)
test_y=np_utils.to_categorical(Y_test)
train_y=train_y.reshape(train_y.shape[1],train_y.shape[2])
test_y=test_y.reshape(test_y.shape[1],test_y.shape[2])
X_train=X_train.reshape(X_train.shape[0],image_x,image_y,1)
X_test=X_test.reshape(X_test.shape[0],image_x,image_y,1)

In [5]:
print("X_train shape=" + str(X_train.shape))
print("Y_train shape=" + str(train_y.shape))

X_train shape=(70000, 32, 32, 1)
Y_train shape=(70000, 37)


In [6]:
def keras_model(image_x,image_y):
    num_of_classes=37
    model=Sequential()
    model.add(Conv2D(filters=32,kernel_size=(5,5),input_shape=(image_x,image_y,1),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same'))
    model.add(Conv2D(64,(5,5),activation='relu'))
    model.add(MaxPooling2D(pool_size=(5,5),strides=(5,5),padding='same'))
    model.add(Flatten())
    model.add(Dense(num_of_classes,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    filepath="devanagari.h5"
    checkpoint1=ModelCheckpoint(filepath,monitor='val_acc',verbose=1,save_best_only=True,mode='max')
    callbacks_list=[checkpoint1]
    
    return  model,callbacks_list

In [8]:
!pip install callbacks

  Created wheel for callbacks: filename=callbacks-0.3.0-py3-none-any.whl size=5658 sha256=89c5eeb0958416d82c3a08837deb192640092d2ec32d40b37f1406a2cd5eb4af
  Stored in directory: /home/rupeek/.cache/pip/wheels/f6/fe/60/5ff958b7d8dd524e2e12e7a75047e3518ef3ea098b9313d6eb
Successfully built callbacks


In [9]:
model,callbacks_list=keras_model(image_x,image_y)
#model.fit(Xtrain, Ytrain, batch_size = 32, epochs = 100)
# #fit(object, x = NULL, y = NULL, batch_size = NULL, epochs = 10,
#   callbacks = NULL, validation_data = NULL,
#   
model.fit(X_train,train_y,batch_size=64,epochs=3,Train on 70000 samples, validate on 2000 samples
Epoch 1/3
70000/70000 [==============================] - 120s 2ms/step - loss: 0.8020 - acc: 0.7724 - val_loss: 0.3450 - val_acc: 0.8980

Epoch 00001: val_acc improved from -inf to 0.89800, saving model to devanagari.h5
Epoch 2/3
70000/70000 [==============================] - 87s 1ms/step - loss: 0.2422 - acc: 0.9263 - val_loss: 0.2332 - val_acc: 0.9335

Epoch 00002: val_acc improved from 0.89800 to 0.93350, saving model to devanagari.h5
Epoch 3/3
70000/70000 [==============================] - 67s 951us/step - loss: 0.1583 - acc: 0.9529 - val_loss: 0.1837 - val_acc: 0.9495

Epoch 00003: val_acc improved from 0.93350 to 0.94950, saving model to devanagari.h5
CNN Error : 5.05%
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
conv2d_1 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 37)                9509      
=================================================================
Total params: 61,605
Trainable params: 61,605callbacks=callbacks_list,validation_data=[X_test,test_y])

scores=model.evaluate(X_test,test_y,verbose=0)
print("CNN Error : %.2f%%" %(100-scores[1]*100))
print_summary(model)
model.save('devanagari.h5')

Epoch 1/3
1094/1094 [==============================] - ETA: 0s - loss: 0.8114 - accuracy: 0.7698

ValueError: in user code:

    /home/rupeek/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1224 test_function  *
        return step_function(self, iterator)
    /home/rupeek/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1215 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/rupeek/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/rupeek/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/rupeek/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/rupeek/anaconda3/lib/python3.7/site-packages/tensorflow/p